In [1]:
import warnings
warnings.filterwarnings('ignore')

# <span style="color:cyan"> Parte 1: Registro de los archivos en la base de datos.

### <span style="color:orange"> Generar una nueva base de datos con la siguiente nomenclatura: apellido_nombre.

In [2]:
import psycopg2

database_name = 'sepulveda_rodrigo'

# Datos conexión
host='localhost'
dbname='sepulveda_rodrigo'
user='postgres'
password='qwertyu8'

# Crear la base de datos
connection = psycopg2.connect(host=host, user=user, password=password)
connection.autocommit = True
cursor = connection.cursor()
cursor.execute(f"CREATE DATABASE {database_name};")

cursor.close()
connection.close()

### <span style="color:orange"> Importar en tablas los archivos train_cupid.csv y test_cupid.csv a un motor Postgres, implementando solo la librería psycopg2. Las tablas deben contener los nombres de las columnas y el total de los registros presente en cada archivo.

In [3]:
# Conexión a la base de datos
connection = psycopg2.connect(host=host, dbname=dbname, user=user, password=password)
# Crear un cursor
cursor = connection.cursor()

In [4]:
import csv

# Nombre de las tablas
tabla_train = 'train_cupid'
tabla_test = 'test_cupid'

# Importar archivo train_cupid.csv
with open('train_cupid.csv', 'r') as file:
    # Leer el archivo CSV
    reader = csv.reader(file)
    
    # Obtener los nombres de las columnas
    header = next(reader)
    
    # Crear la tabla en la base de datos
    column_definitions = ", ".join(f'"{column}" FLOAT' for column in header)
    cursor.execute(f'CREATE TABLE {tabla_train} ({column_definitions})')
    
    # Insertar los registros en la tabla
    for row in reader:
        cursor.execute(f"INSERT INTO {tabla_train} VALUES ({', '.join(['%s'] * len(row))})", row)

connection.commit()

In [5]:
# Importar archivo test_cupid.csv
with open('test_cupid.csv', 'r') as file:
    # Leer el archivo CSV
    reader = csv.reader(file)
    
    # Obtener los nombres de las columnas
    header = next(reader)
    
    # Crear la tabla en la base de datos
    column_definitions = ", ".join(f'"{column}" FLOAT' for column in header)
    cursor.execute(f'CREATE TABLE {tabla_test} ({column_definitions})')
    
    # Insertar los registros en la tabla
    for row in reader:
        cursor.execute(f"INSERT INTO {tabla_test} VALUES ({', '.join(['%s'] * len(row))})", row)

connection.commit()

# Cerrar cursor y conexión
cursor.close()
connection.close()

# <span style="color:cyan"> Parte 2: Entrenamiento de modelos

### <span style="color:orange"> Ingestar la tabla de training mediante psycopg2 para el posterior entrenamiento del modelo.

In [6]:
# Conexión a la base de datos
connection = psycopg2.connect(host=host, dbname=dbname, user=user, password=password)
# Crear un cursor
cursor = connection.cursor()

In [7]:
import pandas as pd

# generación de consulta en el cursor
cursor.execute(f'SELECT * FROM {tabla_train}')
data_train = cursor.fetchall()

# Cerrar cursor y conexión
cursor.close()
connection.close()

# transformación de consulta en una lista antes de ingresarla como un pd.DataFrame
df_train = pd.DataFrame(list(data_train))

# renombro las columnas
df_train.columns = header
df_train

# esto mismo se puede lograr con la siguiente línea de código pero está soportado de mejor manera con SQLAlchemy 
# pd.read_sql(sql='SELECT * FROM train_cupid;', con=connection)

,age,height,virgo,taurus,scorpio,pisces,libra,leo,gemini,aries,...,orientation_straight,sex_m,smokes_sometimes,smokes_trying to quit,smokes_when drinking,smokes_yes,body_type_overweight,body_type_regular,education_high_school,education_undergrad_university
0,35.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,38.0,68.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,23.0,71.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,29.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,29.0,67.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20076,33.0,63.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20077,22.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
20078,28.0,64.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20079,31.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### <span style="color:orange"> Entrenar los siguientes modelos (sin necesidad de ajustar por hiper parámetros):
    - GradientBoostingClassifier
    - AdaBoostClassifer
    - RandomForestClassifier
    - SVC
    - DecisionTreeClassifier
    - LogisticRegression
    - BernoulliNB.

In [8]:
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB

In [9]:
df_train_copy = df_train.copy()

# vectores objetivo
targets = ['available', 'seeing_someone', 'single']

# modelos a implementar
models = {
        'GradientBoostingClassifier': GradientBoostingClassifier(),
        'AdaBoostClassifier': AdaBoostClassifier(),
        'RandomForestClassifier': RandomForestClassifier(),
        'SVC': SVC(),
        'DecisionTreeClassifier': DecisionTreeClassifier(),
        'LogisticRegression': LogisticRegression(max_iter=5000),
        'BernoulliNB': BernoulliNB()
    }

    Se ingresó el hiperparámetro max_iter=5000 en 'LogisticRegression' ya que al ajustar el modelo arrojaba error y una de las soluciones era implementar este hiperparámetro.

### <span style="color:orange"> Serializar el objeto y preservarlo por cada combinación de modelo entrenado y vector objetivo.

In [10]:
import pickle

serialized_models = {}

# matriz atributos entrenamiento
X_train = df_train_copy.drop(targets, axis=1)

# Entrenar los modelos y serializarlos por cada combinación de modelo y vector objetivo
for target in targets:

    # vector objetivo entrenamiento
    y_train = df_train_copy[target]

    # Inicializar y entrenar los modelos
    for model_name, model in models.items():
        model.fit(X_train, y_train)
        
        # Serializar el objeto del modelo entrenado
        pickle.dump(model, open('Modelo_' + model_name + '_' + target + '.pkl', 'wb')) 

        # guardado de nombre de modelos en diccionario
        serialized_models['Modelo_' + model_name + '_' + target + '.pkl'] = model

# <span style="color:cyan"> Parte 3: Exportación de predicciones

### <span style="color:orange"> Ingestar la tabla de testing mediante psycopg2 para la posterior predicción del modelo.

In [11]:
# Conexión a la base de datos
connection = psycopg2.connect(host=host, dbname=dbname, user=user, password=password)
# Crear un cursor
cursor = connection.cursor()

In [12]:
# generación de consulta en el cursor
cursor.execute(f'SELECT * FROM {tabla_test}')
data_test = cursor.fetchall()

# Cerrar cursor y conexión
cursor.close()
connection.close()

# transformación de consulta en una lista antes de ingresarla como un pd.DataFrame
df_test = pd.DataFrame(list(data_test))

# renombro las columnas
df_test.columns = header
df_test

,age,height,virgo,taurus,scorpio,pisces,libra,leo,gemini,aries,...,orientation_straight,sex_m,smokes_sometimes,smokes_trying to quit,smokes_when drinking,smokes_yes,body_type_overweight,body_type_regular,education_high_school,education_undergrad_university
0,22.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,32.0,65.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,24.0,67.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,29.0,62.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,39.0,65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19938,48.0,73.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19939,52.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
19940,59.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
19941,24.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### <span style="color:orange"> En base a los objetos serializados, predecir y evaluar cuatro queries específicas:
    - Query 1: 'atheism', 'asian', 'employed', 'pro_dogs', 'chinese'.
    - Query 2: 'income_over_75', 'french', 'german','orientation_straight', 'new york'.
    - Query 3: 'education_undergrad_university', 'body_type_regular', 'pro_dogs', 'employed'.
    - Query 4: 'taurus', 'indian', 'washington', 'income_between_50_75', 'hinduism'.

In [13]:
df_test_copy = df_test.copy()

# matriz atributos datos validación
X_test = df_test_copy.drop(targets, axis=1)

# y_test de cada vector objetivo
y_test_available = df_test_copy['available']
y_test_seeing_someone =df_test_copy['seeing_someone']
y_test_single =df_test_copy['single']

In [14]:
# función para calcular las predicciones
def predictions(model, X_test):
    yhat_temp = model.predict(X_test)
    return yhat_temp

In [15]:
# obtención de predicciones de los modelos serializados
y_hat_dict = {}

for key in serialized_models.keys():
    modelo_pickle_ajustado = pickle.load(open(key,"rb"))
    y_hat_test = predictions(modelo_pickle_ajustado, X_test)
    y_hat_dict[key.replace('Modelo_', '').replace('.pkl', '_predictions')] = y_hat_test

In [16]:
# dataframe que contiene las predicciones para los 21 modelos obtenidos
df_yhat_test = pd.DataFrame(y_hat_dict)

In [17]:
# concatenación df_yhat_test con X_test
df_concat = pd.concat([X_test, df_yhat_test], axis=1)

In [18]:
from sklearn.metrics import classification_report

# función que imprime los classification report para cada vector objetivo
def classif_report(target):
    if target == 'available':
        print('*'*13,'Vector objetivo available','*'*13, '\n')
        for prediction in df_yhat_test.iloc[:,:7]:
            print(f'Métricas para {prediction.replace("_predictions", "")}:')
            print(classification_report(y_test_available, df_yhat_test[prediction]), '\n', '-'*60, '\n')

    elif target == 'seeing_someone':
        print('*'*13,'Vector objetivo seeing_someone','*'*13, '\n')
        for prediction in df_yhat_test.iloc[:, 7:14]:
            print(f'Métricas para {prediction.replace("_predictions", "")}:')
            print(classification_report(y_test_seeing_someone, df_yhat_test[prediction]), '\n', '-'*60, '\n')

    elif target == 'single':
        print('*'*14,'Vector objetivo single','*'*14, '\n')
        for prediction in df_yhat_test.iloc[:, 14:]:
            print(f'Métricas para {prediction.replace("_predictions", "")}:')
            print(classification_report(y_test_single, df_yhat_test[prediction]), '\n', '-'*60, '\n')

In [19]:
# classification report para los vectores objetivos
classif_report('available')
classif_report('seeing_someone')
classif_report('single')

************* Vector objetivo available ************* 

Métricas para GradientBoostingClassifier_available:
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     19163
         1.0       0.00      0.00      0.00       780

    accuracy                           0.96     19943
   macro avg       0.48      0.50      0.49     19943
weighted avg       0.92      0.96      0.94     19943
 
 ------------------------------------------------------------ 

Métricas para AdaBoostClassifier_available:
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98     19163
         1.0       0.00      0.00      0.00       780

    accuracy                           0.96     19943
   macro avg       0.48      0.50      0.49     19943
weighted avg       0.92      0.96      0.94     19943
 
 ------------------------------------------------------------ 

Métricas para RandomForestClassifier_available:
              pr

In [20]:
queries = [
    ['atheism', 'asian', 'employed', 'pro_dogs', 'chinese'],
    ['income_over_75', 'french', 'german', 'orientation_straight', 'new york'],
    ['education_undergrad_university', 'body_type_regular', 'pro_dogs', 'employed'],
    ['taurus', 'indian', 'washington', 'income_between_50_75', 'hinduism']
]

In [21]:
# función que retorna los resultados para cada query
def result_query(query, model_target_yhat):
    result_query = df_concat.groupby(query)[model_target_yhat].mean()
    return result_query

In [22]:
# se guardan en una lista los resultados por query
resultados = []
for query_i in queries:
    for y_hat in df_yhat_test.columns:
        resultados.append(result_query(query=query_i, model_target_yhat=y_hat))

In [23]:
# lista para los nombres de las tablas que crearé en sql
nombres_de_tablas=[]

queries_name = ['query_1', 'query_2', 'query_3', 'query_4']
for query in queries_name:
    for i in range(0, 21):
        nombres_de_tablas.append(resultados[i].name+'_'+query)

In [24]:
from sqlalchemy import create_engine

# conexion a postgres usando sqlalchemy
engine = create_engine('postgresql://' + user + ':' + password + '@' + host + '/' + dbname)

def tabla_a_sql(indice, nombre_de_la_tabla):
    # convierto tabla a dataframe
    temp_df_resultado_i = resultados[indice].to_frame()

    # ahora convierto de dataframe a tabla en sql
    return temp_df_resultado_i.to_sql(nombre_de_la_tabla , con=engine, if_exists="replace")

In [25]:
# Creación de tablas en sql
for index, value in enumerate(nombres_de_tablas):
    tabla_a_sql(index, value)